In [51]:
import torch
import json
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample


In [58]:
def readTrainData(path):
    trainDataset=[]
    with open(path,'rb')as file:
        STSdata=json.load(file)
    
    for item in STSdata:
        score=float(item['labels']['label'])/5.0

        inputEx=InputExample(
        texts=[item["sentence1"],item["sentence2"]],
        label=score
        )
        trainDataset.append(inputEx)
    
    return trainDataset

In [59]:
modelName="klue/bert-base"
embeddingModelName=models.Transformer(modelName)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [95]:
BATCH_SIZE=32
EPOCH=4
outputPath=(modelName+"_epoch-"+str(EPOCH)).replace("/","-")
outputPath='./outputs/'+outputPath

In [96]:
outputPath

'./outputs/klue-bert-base_epoch-4'

In [97]:
#pooling, mean 사용예정
pooler = models.Pooling(
    embeddingModelName.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [98]:
model = SentenceTransformer(modules=[embeddingModelName, pooler])

In [99]:
dataName="./dataset/klue-sts-v1.1_train.json"
trainDataset=readTrainData(dataName)

In [100]:
trainDataLoader=DataLoader(
trainDataset,
shuffle=True,
batch_size=BATCH_SIZE
)
trainLoss=losses.CosineSimilarityLoss(model=model)

In [101]:
evaluator=EmbeddingSimilarityEvaluator.from_input_examples(
trainDataset,
name="sts-vali"
)

In [102]:
model.fit(
train_objectives=[(trainDataLoader, trainLoss)],
    evaluator=evaluator,
    epochs=EPOCH,
    evaluation_steps=1000,
    output_path=outputPath
 )

C:\Users\DESKTOP\anaconda3\envs\a2\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]